# What might we do differently?

## Purpose:
To briefly discuss the availability of subjective and objective data relationship analytics. 

## Background:
*LLM's* and *vector* search are great for the subjective assessment of data relationshsips - they identify collections that are symantically **"alike"**.  This enables LLM's to be *creative* which is also why they get things wrong.
Graph databases, specifically *knowledge graphs*, are wholly objective in their inferences through the data relationships.  Wikidata is a graph database that sits behind a large proprtion of wikipedia. 

## Method:
Use *Langchain* to orchestrate GPT-4 to write some code to construct an API call (and query) into Wikidata, to answer a specific questions.  Langchain will the manage the execution of the code, to return the response back to GPT-4 to format the layout.   

---

### What is a graph database?

<figure>
     <a href="https://link.springer.com/article/10.1007/s10639-023-11664-1/figures/3">
          <img src="https://media.springernature.com/full/springer-static/image/art%3A10.1007%2Fs10639-023-11664-1/MediaObjects/10639_2023_11664_Fig3_HTML.png"
               alt="image of a simple graph"
               height=600px>
     </a>
     <figcaption>A simple graph, used by the Metropoltan Museum of Art, to learn of the link between Madame X and the dress worn by Rita Hayworth in Gilda
</figure>




<figure>
     <a href="https://www.wikidata.org/wiki/Wikidata:Tools/Visualize_data#/media/File:Wikidata_Tree_Builder.png">
          <img src="https://upload.wikimedia.org/wikipedia/commons/1/1b/Wikidata_Tree_Builder.png"
               alt="image of a detailed graph"
               height=600px>
     </a>
     <figcaption>A dynamically generated graph, from  data in wikidata, showing the subclasses to two levels (and interconnects) of occupations using *artist* as the root
</figure>


Demonstrate the use of wikidata to get to some data, but do so using langchain to have GPT-4 construct Python and query and pass to PythonREPL to execute and then format neatly


### SPARQL to fetch the Shakespeare birth and dates and locations from wikidata

URL = [https://query.wikidata.org/](https://query.wikidata.org/#%20%20%20%20SELECT%20%3FbirthDate%20%3FdeathDate%20%3FbirthPlaceLabel%20%3FdeathPlaceLabel%20WHERE%20%7B%0A%20%20%20%20%23%20William%20Shakespeare%27s%20entity%20in%20Wikidata%0A%20%20%20%20BIND%28wd%3AQ692%20AS%20%3Fshakespeare%29%0A%0A%20%20%20%20%23%20Retrieve%20birth%20and%20death%20dates%0A%20%20%20%20%3Fshakespeare%20wdt%3AP569%20%3FbirthDate%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP570%20%3FdeathDate%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP19%20%3FbirthPlace%3B%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP20%20%3FdeathPlace.%20%20%20%20%20%20%20%0A%0A%20%20%20%20%23%20Get%20the%20labels%20of%20the%20birth%20and%20death%20places%20in%20English%2C%20as%20it%20returns%20as%20codes%0A%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%0A%20%20%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%0A%20%20%20%20%20%20%20%20%3FbirthPlace%20rdfs%3Alabel%20%3FbirthPlaceLabel.%0A%20%20%20%20%20%20%20%20%3FdeathPlace%20rdfs%3Alabel%20%3FdeathPlaceLabel.%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%7D)


    SELECT ?birthDate ?deathDate ?birthPlaceLabel ?deathPlaceLabel WHERE {
    # William Shakespeare's entity in Wikidata
    BIND(wd:Q692 AS ?shakespeare)

    # Retrieve birth and death dates
    ?shakespeare wdt:P569 ?birthDate;
                wdt:P570 ?deathDate;
                wdt:P19 ?birthPlace;       
                wdt:P20 ?deathPlace.       

    # Get the labels of the birth and death places in English, as it returns as codes
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
        ?birthPlace rdfs:label ?birthPlaceLabel.
        ?deathPlace rdfs:label ?deathPlaceLabel.
        }
    }


### Example

Demonstrate the use of wikidata to source data, but do so using langchain to have GPT-4 construct Python and query and pass to PythonREPL to execute and then format neatly


In [2]:
import requests
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI

#from langchain.llms.openai import OpenAIChat
pre_request = "Using Wikidata. "
post_request = "  The requests library is already installed."
agent_executor = create_python_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-4-0613"), 
    tool=PythonREPLTool(),
    verbose=True
)

request = "Locate and printout the dates and locations of the birth and death of William Shakespeare."
# request = "Locate and printout the papers and dates of publication that were jointly worked on by Albert Einstein and Jakob Laub."
agent_executor.run(pre_request + request + post_request)



> Entering new  chain...
I need to use the requests library to send a GET request to the Wikidata API. The API endpoint for Wikidata is "https://www.wikidata.org/w/api.php". I need to pass the action, format, and the search parameters to the API endpoint. The action will be 'wbgetentities', the format will be 'json', and the search parameter will be 'William Shakespeare'. After getting the response, I need to parse the JSON data to get the birth and death dates and locations of William Shakespeare.
Action: Python REPL
Action Input: 
```python
import requests

# Define the API endpoint
url = "https://www.wikidata.org/w/api.php"

# Define the parameters for the API request
params = {
    "action": "wbgetentities",
    "format": "json",
    "titles": "William_Shakespeare",
    "props": "claims",
    "sitefilter": "enwiki"
}

# Send the GET request
response = requests.get(url, params=params)

# Parse the JSON data from the response
data = response.json()

# Get the entity ID of William S

'William Shakespeare was born on April 23, 1564 and died on April 23, 1616. Both his birth and death took place in Stratford-upon-Avon.'